# **XGBoost Regressor**

Este notebook tiene como objetivo entrenar, analizar y optimizar un modelo de **XGBoost Regressor** aplicado al dataset de **Esperanza de Vida**.

**¿Por qué XGBoost?**  
- Es un algoritmo basado en **árboles de decisión** que utiliza la técnica de **boosting** (aprendizaje secuencial), lo que lo hace muy potente para datos tabulares.  
- Suele obtener mejores resultados que modelos más simples (como Regresión Lineal o Árboles de Decisión) y maneja bien relaciones no lineales y variables con distintas escalas.  
- Incluye regularización para evitar **overfitting**, algo que puede ser un problema en Decision Trees simples.  

**Qué haremos en este notebook:**  
1. Importar librerías y cargar los datasets procesados.  
2. Entrenar un modelo **baseline** de XGBoost y evaluar métricas (RMSE, MAE, R²).  
3. Analizar **overfitting** mediante curvas de validación.  
4. Visualizar la **importancia de variables** en el modelo.  
5. Optimizar hiperparámetros con **Optuna** para mejorar el rendimiento.  
6. Comparar el modelo baseline con el optimizado.  
7. Concluir sobre la utilidad de XGBoost en este problema y compararlo brevemente con Decision Tree y Random Forest.  

Este notebook complementa al de **DecisionTree_RandomForest.ipynb**, profundizando en el uso de un algoritmo de boosting más avanzado.


## **Paso 1. Importar librerías**

En este bloque importamos todas las librerías necesarias:  

- **pandas** para manipulación de datos.  
- **train_test_split** para dividir el dataset en entrenamiento y validación.  
- **XGBRegressor** de `xgboost` como modelo principal.  
- **métricas de sklearn** para evaluar el rendimiento (RMSE, MAE, R²).  
- **matplotlib** y **seaborn** para visualizaciones.  
- **numpy** para operaciones matemáticas.  

Dejamos preparado también el entorno para trabajar con **Optuna**, que utilizaremos más adelante en la optimización de hiperparámetros.


In [ ]:
# ===================================
# 1. Importar librerías
# ===================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

# Modelo principal
from xgboost import XGBRegressor, plot_importance

# Para optimización
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_parallel_coordinate


## **Paso 2. Carga de datos procesados y no escalados**

En este bloque importamos las librerías necesarias para entrenar el modelo **XGBoost** 
y cargamos los datasets procesados.

👉 Usaremos los datos **no escalados**, ya que XGBoost (al igual que Random Forest) 
no requiere que las variables estén normalizadas o estandarizadas.  
El dataset ya está limpio y preprocesado en el notebook `EDA_processed.ipynb`.  

También dividimos los datos en **train** y **validación** para evaluar el modelo.


In [ ]:
# ===================================
# 2. Cargar datos procesados y no escalados
# ===================================

# --- Cargar datasets procesados ---
X_ns = pd.read_csv("../data/processed/features_no_scaling.csv")
y = pd.read_csv("../data/processed/target_y.csv").squeeze()  # lo convertimos a Serie

# División train / validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X_ns, y, test_size=0.2, random_state=42
)

print("Tamaño entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño validación:", X_valid.shape, y_valid.shape)


## **Paso 3. Entrenamiento y evaluación del modelo XGBoost (baseline)**

En este bloque entrenamos un **XGBoost Regressor** con parámetros por defecto.  
Este modelo pertenece a la familia de **boosting**, es decir, combina árboles de decisión de forma secuencial, donde cada árbol corrige los errores del anterior.  

Evaluamos las métricas principales:  

- **RMSE (Root Mean Squared Error):** mide el error promedio penalizando más los errores grandes.  
- **MAE (Mean Absolute Error):** mide el error promedio sin penalizar tanto los valores extremos.  
- **R² (Coeficiente de determinación):** indica cuánto del comportamiento de la variable objetivo es explicado por el modelo (1 = perfecto, 0 = no explica nada).  

Estos valores nos darán una primera referencia de qué tan bien se comporta el modelo **sin ajustar hiperparámetros**.

### 📊 Visualización: valores reales vs. predichos (XGBoost baseline)

En este gráfico comparamos los **valores reales de la esperanza de vida** (eje X) frente a los **valores predichos por el modelo** (eje Y).

- La **línea diagonal en rojo** representa la situación ideal: todas las predicciones coinciden exactamente con los valores reales.
- Los **puntos azules** son las predicciones reales del modelo.

👉 Interpretación:
- Cuanto más cerca estén los puntos de la línea roja, mejor está funcionando el modelo.
- Si los puntos se dispersan mucho lejos de la línea, significa que el modelo está cometiendo errores grandes en esas observaciones.


In [ ]:
# ===================================
# 3. Entrenar y evaluar modelo (baseline)
# ===================================
from xgboost import XGBRegressor

# Modelo baseline con parámetros por defecto
xgb = XGBRegressor(
    random_state=42,
    n_estimators=100,   # número de árboles (default suele ser 100)
    learning_rate=0.1,  # tasa de aprendizaje
    max_depth=3         # profundidad máxima de cada árbol
)

xgb.fit(X_train, y_train)

# Predicciones
preds_xgb = xgb.predict(X_valid)

# Métricas
rmse = np.sqrt(mean_squared_error(y_valid, preds_xgb))
mae = mean_absolute_error(y_valid, preds_xgb)
r2 = r2_score(y_valid, preds_xgb)

print("📊 XGBoost (baseline)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")

# ===================================
# Visualización: valores reales vs. predichos
# ===================================

plt.figure(figsize=(6,6))
sns.scatterplot(x=y_valid, y=preds_xgb, alpha=0.6)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], color="red", lw=2)
plt.xlabel("Valores reales")
plt.ylabel("Valores predichos")
plt.title("XGBoost - Valores reales vs. predichos")
plt.show()



## **Paso 4. Diagnóstico de residuos**

Antes de evaluar posibles problemas de sobreajuste, analizamos los **residuos del modelo**:

- Los **residuos** son las diferencias entre los valores reales (`y_valid`) y las predicciones del modelo (`preds`).
- Un buen modelo debería mostrar residuos **centrados en 0** y sin patrones claros.

👉 En este bloque:
1. Graficamos un **Histograma de residuos:** nos muestra cómo se distribuyen los errores.  
  - Lo ideal es que los residuos estén centrados en 0 y tengan una forma simétrica.  
  - Una gran asimetría o colas largas pueden indicar que el modelo falla en ciertos rangos.  

2. Graficamos **Dispersión `y_valid vs. predicciones`:** permite ver si hay **patrones en los errores**, comprobamos si las predicciones siguen bien la diagonal ideal.
  - Si los puntos están alineados en torno a la diagonal, significa que el modelo predice bien.  
  - Si vemos un patrón en forma de curva o zonas donde el modelo sistemáticamente se equivoca (ej. subestima en valores altos), puede haber sesgos o falta de complejidad.  

A diferencia de la regresión lineal, aquí **no buscamos una relación perfectamente lineal**, sino que el modelo capture adecuadamente las variaciones sin dejar patrones evidentes en los residuos.
 
### Interpretación:
- Si los residuos se concentran alrededor de 0 → el modelo generaliza bien.
- Si hay colas largas o muchos residuos grandes → el modelo está fallando en algunos casos.
- Si los residuos muestran un patrón (ej. tendencia en forma de curva) → puede que falte complejidad o que el modelo esté sesgado.


In [ ]:
# ===================================
# 4. Diagnóstico de residuos (XGBoost)
# ===================================

# Cálculo de residuos
residuos = y_valid - preds_xgb

# --- Gráfico 1: valores reales (y_valid) vs predicciones ---
plt.figure(figsize=(7,5))
sns.scatterplot(x=y_valid, y=preds_xgb, alpha=0.6)
plt.plot([y_valid.min(), y_valid.max()],
         [y_valid.min(), y_valid.max()],
         color="red", linestyle="--")
plt.xlabel("Valores reales (y_valid)")
plt.ylabel("Predicciones")
plt.title("Predicciones vs. Valores reales (XGBoost)")
plt.show()

# --- Gráfico 2: Residuos vs predicciones ---
plt.figure(figsize=(7,5))
sns.scatterplot(x=preds_xgb, y=residuos, alpha=0.6)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Predicciones")
plt.ylabel("Residuos")
plt.title("Residuos vs. Predicciones (XGBoost)")
plt.show()

# --- Gráfico 3: Distribución de residuos ---
plt.figure(figsize=(7,5))
sns.histplot(residuos, bins=30, kde=True, color="steelblue")
plt.axvline(0, color="red", linestyle="--")
plt.xlabel("Residuos: Error (y_valid - predicción)")
plt.ylabel("Frecuencia")
plt.title("Distribución de residuos (XGBoost)")
plt.show()


## **Paso 5. Overfitting en XGBoost: curva de aprendizaje con max_depth**

XGBoost, al igual que los árboles de decisión y Random Forest, puede sobreajustar fácilmente si los árboles son muy profundos.

En este bloque:
- Variamos el hiperparámetro `max_depth` (profundidad máxima de cada árbol).
- Calculamos el **RMSE en entrenamiento y validación** para cada valor.
- Graficamos ambas curvas para ver la diferencia.

- **Entrenamiento (RMSE Train):** mide qué tan bien el modelo memoriza los datos vistos.  
- **Validación (RMSE Valid):** mide qué tan bien generaliza el modelo en datos no vistos.  

📊 Interpretación:
- Si el error de **train** es muy bajo (baja mucho la curva de **entrenamiento**) y el de **validación** empeora o dja de mejorar (es muy alto) → hay **overfitting**.  
- Si ambos errores (o curvas) son altos → el modelo está **underfitting** (falta de complejidad).  
- Lo ideal, el punto óptimo, está en encontrar un punto intermedio donde los dos errores (o las dos curvas) sean bajos y cercanos.



In [ ]:
# ===================================
# Paso 5. Overfitting en XGBoost
# Curva de aprendizaje con max_depth
# ===================================

from xgboost import XGBRegressor

train_errors, valid_errors = [], []
depths = range(1, 16)  # probamos de profundidad 1 a 16

for d in depths:
    model = XGBRegressor(
        random_state=42,
        n_estimators=200,     # número de árboles
        learning_rate=0.1,    # tamaño del paso
        max_depth=d,
        n_jobs=-1          # usar todos los núcleos disponibles
    )
    model.fit(X_train, y_train)
    
    preds_train = model.predict(X_train)
    preds_valid = model.predict(X_valid)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, preds_train))
    valid_rmse = np.sqrt(mean_squared_error(y_valid, preds_valid))
    
    train_errors.append(train_rmse)
    valid_errors.append(valid_rmse)

plt.figure(figsize=(8,5))
plt.plot(depths, train_errors, label="Train RMSE", marker="o")
plt.plot(depths, valid_errors, label="Valid RMSE", marker="o")
plt.xlabel("Profundidad del árbol (max_depth)")
plt.ylabel("RMSE")
plt.title("Curva de aprendizaje - Overfitting en XGBoost")
plt.legend()
plt.show()


## **Paso 6. Optimización de hiperparámetros con Optuna**

Hasta ahora hemos probado XGBoost con parámetros por defecto y variando solo `max_depth`.  
Pero este modelo tiene **muchos hiperparámetros clave** que afectan al rendimiento y al riesgo de **overfitting**.  

Algunos de los más importantes:  
- `n_estimators`: número de árboles que se entrenan (bagging).  
- `max_depth`: profundidad máxima de cada árbol.  
- `learning_rate`: tasa de aprendizaje para cada actualización de boosting.  
- `subsample`: fracción de filas que se muestrean al entrenar cada árbol.  
- `colsample_bytree`: fracción de columnas (features) que se usan por árbol.  
- `gamma`: regularización de la división de nodos (controla la complejidad).  
- `reg_lambda`: regularización L2 para evitar overfitting.  

👉 Buscar manualmente todos estos parámetros sería muy costoso.  
Por eso usamos **Optuna**, una librería de optimización automática, que explora el espacio de parámetros y encuentra las combinaciones que minimizan el error de validación (aquí usaremos **RMSE**).  

Este paso nos permitirá obtener un XGBoost mucho más competitivo y comparar con Random Forest de forma justa.


In [ ]:
# ===================================
# 6. Optuna: optimización de hiperparámetros
# ===================================
import optuna
from xgboost import XGBRegressor

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "random_state": 42,
        "eval_metric": "rmse"   # 
    }
    
    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    
    preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    return rmse

# Crear estudio de optimización
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)  # número de iteraciones (aumentar si tienes tiempo)

print("📊 Mejores parámetros encontrados:")
print(study.best_params)
print(f"Mejor RMSE validación: {study.best_value:.3f}")


## **Paso 7. Evaluación tras la optimización**

Evaluación del mejor modelo con los parámetros encontrador por Optuna.

Aunque Optuna se ha encargado de **minimizar RMSE**, no nos quedamos solo con esa métrica.  
Una vez obtenido el mejor conjunto de hiperparámetros, volvemos a entrenar el modelo y lo evaluamos con **RMSE, MAE y R²**:

- **RMSE**: penaliza más los errores grandes, útil para medir precisión global.  
- **MAE**: mide el error medio absoluto, más robusto frente a valores atípicos.  
- **R²**: proporción de la varianza de la variable objetivo explicada por el modelo.  

Esto nos permite comparar con el resto de algoritmos bajo un mismo criterio.



In [ ]:
# ===================================
# 7. Evaluación del mejor modelo Optuna
# ===================================

best_params = study.best_params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

# Predicciones
preds_optuna = best_model.predict(X_valid)

# Métricas
rmse = np.sqrt(mean_squared_error(y_valid, preds_optuna))
mae = mean_absolute_error(y_valid, preds_optuna)
r2 = r2_score(y_valid, preds_optuna)

print("📊 XGBoost optimizado con Optuna")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


## **Paso 8. Convergencia de Optuna**

Optuna permite visualizar cómo evoluciona la métrica objetivo (en este caso, **RMSE**) a lo largo de los *trials*.

👉 Si la curva desciende rápidamente y luego se estabiliza, significa que el algoritmo encontró buenos hiperparámetros pronto y después solo afina ligeramente.

👉 Si la curva sigue bajando mucho trial tras trial, quizá convendría aumentar el número de *trials*.

Este gráfico nos ayuda a diagnosticar:
- **Rapidez de convergencia** del proceso de optimización.  
- Si los parámetros óptimos encontrados son estables o podrían mejorar con más iteraciones.





In [ ]:
# ===================================
# 8. Visualización de convergencia de Optuna
# ===================================


fig = plot_optimization_history(study)
fig.show()


## **Interpretación del gráfico de convergencia de Optuna**

El gráfico de **convergencia de Optuna** nos muestra cómo evoluciona el error del modelo (RMSE en validación) a medida que se prueban diferentes combinaciones de hiperparámetros.



###  ¿Qué representa?
- **Eje X (iteraciones / trials):** cada prueba realizada con un conjunto distinto de hiperparámetros.  
- **Eje Y (RMSE en validación):** el valor de la métrica que tratamos de minimizar en cada trial.  
- **Puntos / línea azul:** error alcanzado en cada prueba.  
- **Línea roja (si aparece):** el mejor valor encontrado hasta ese momento (incumbent curve).  


###  Cómo interpretarlo
1. **Descenso rápido inicial** → significa que Optuna encontró enseguida parámetros mucho mejores que los iniciales.  
2. **Meseta después de varios trials** → indica que el modelo ya alcanzó un buen rendimiento y no mejora con más pruebas.  
3. **Picos hacia arriba en algunos trials** → normales, Optuna explora combinaciones que a veces rinden peor.  



###  Qué nos dice sobre nuestro modelo
- Si la curva converge rápido → el modelo es **estable y fácil de ajustar**.  
- Si mejora poco a poco tras muchos trials → puede valer la pena aumentar `n_trials`.  
- Si la curva nunca baja mucho → el modelo tiene limitaciones estructurales (no basta con ajustar parámetros).  

En nuestro caso, Optuna alcanzó un **RMSE ≈ 1.565**, lo que indica un buen ajuste de hiperparámetros. 


## **Paso 9. Importancia de hiperparámetros con Optuna**

Además de encontrar los mejores valores, Optuna nos permite analizar **qué hiperparámetros fueron más relevantes** para reducir el error del modelo.  

- Esta visualización muestra **la contribución relativa de cada hiperparámetro** en el rendimiento (RMSE).  
- Cuanto más alta sea la importancia, más influyó ese parámetro en el resultado.  
- Esto nos ayuda a entender **qué parámetros merecen más atención** en futuros ajustes.  


**Significado de los principales hiperparámetros de XGBoost**

Optuna buscó los mejores valores para los siguientes hiperparámetros:

- **n_estimators** → número de árboles en el ensemble.  
  - Valores altos aumentan la capacidad del modelo, pero también el tiempo de entrenamiento y riesgo de sobreajuste.  

- **max_depth** → profundidad máxima de cada árbol.  
  - Árboles más profundos capturan interacciones complejas, pero pueden sobreajustar.  

- **learning_rate (eta)** → cuánto se corrige cada nuevo árbol respecto al error anterior.  
  - Valores bajos hacen que el modelo aprenda más despacio pero de forma más estable.  

- **subsample** → fracción de muestras que usa cada árbol al entrenarse.  
  - Reduce sobreajuste al introducir aleatoriedad (ejemplo: 0.8 = usa el 80% de los datos en cada árbol).  

- **colsample_bytree** → fracción de variables que se usan en cada árbol.  
  - Similar al subsample, pero a nivel de columnas (features).  

- **gamma** → penalización mínima de reducción de pérdida para hacer una partición.  
  - Valores altos generan árboles más conservadores (menos divisiones).  

- **reg_lambda (L2 regularization)** → fuerza de la regularización en los pesos de los árboles.  
  - Ayuda a reducir sobreajuste y estabilizar el modelo.



In [ ]:
# ===================================
# 9. Importancia de hiperparámetros con Optuna
# ===================================
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show()


## **Paso 10. Importancia de las variables en XGBoost**

Al igual que Random Forest, XGBoost nos permite medir **qué variables aportan más información para reducir el error**.  
Esto se calcula a partir de la frecuencia e impacto con que una variable aparece en los nodos de decisión.

El gráfico mostrará las **20 variables más influyentes** en la predicción de la esperanza de vida.


## 📊 Interpretación del gráfico de importancia de variables  

El gráfico muestra las **20 variables más influyentes** en el modelo XGBoost:  

- **Eje Y** → nombre de las variables predictoras.  
- **Eje X** → valor de importancia asignado por el modelo.  
- **Barras más largas** → indican que esa variable contribuyó más a reducir el error en los árboles.  

**Cómo leerlo:**  
1. Las primeras variables del ranking son las que el modelo usó con más frecuencia y mayor efecto en las divisiones.  
2. Variables con barras cortas tuvieron un impacto mucho menor en las predicciones.  
3. La comparación permite identificar cuáles son los **principales factores asociados a la esperanza de vida** en este dataset.  

**Qué nos dice:**  
- Permite detectar patrones: por ejemplo, si “Adult Mortality” o “GDP” aparecen arriba, sabemos que son variables determinantes.  
- Ofrece información **explicable y accionable**, útil para responsables de políticas de salud o análisis económico.  


## 🔎 Diferencias en la importancia de variables entre XGBoost y Random Forest

Aunque ambos modelos están basados en **árboles de decisión**, el cálculo de la importancia de las variables no es idéntico:

1. **Random Forest (bagging):**  
   - Promedia muchos árboles independientes entrenados sobre subconjuntos aleatorios de datos y variables.  
   - La importancia se calcula según cuánto reduce la **impureza (varianza)** en los nodos a lo largo de todos los árboles.  
   - Tiende a repartir importancia entre variables correlacionadas.

2. **XGBoost (boosting):**  
   - Construye los árboles de forma **secuencial**, corrigiendo errores de predicciones anteriores.  
   - Da más importancia a las variables que ayudan a mejorar el ajuste en cada iteración.  
   - Puede concentrar la relevancia en unas pocas variables clave y dejar otras con menor peso.

📊 **Por qué no coinciden las top 20 variables:**  
- Cada modelo detecta y prioriza relaciones distintas.  
- Con variables correlacionadas, Random Forest reparte importancia, mientras que XGBoost selecciona la más eficaz para reducir error.  
- Esto no significa que uno sea “mejor” que otro, sino que reflejan estrategias distintas.

💡 **Conclusión:**  
Las diferencias en las variables más importantes son **normales**. Ambos modelos coinciden en gran parte en las variables clave, pero difieren en el orden y en cómo asignan la importancia.  
Esto refleja la naturaleza distinta de los métodos de **bagging (Random Forest)** y **boosting (XGBoost)**.



In [ ]:
# ===================================
# 10. Importancia de variables en XGBoost (agrupadas)
# ===================================

def clean_feature_name(name):
    """Agrupa dummies de Country y Status en sus variables originales"""
    if name.startswith("Country_"):
        return "Country"
    if name.startswith("Status_"):
        return "Status"
    return name

# DataFrame de importancias
importances = pd.DataFrame({
    "Variable": X_train.columns,
    "Importancia": model.feature_importances_
})

# Reagrupar importancias
importances["Variable_grouped"] = importances["Variable"].apply(clean_feature_name)
grouped = (
    importances.groupby("Variable_grouped")["Importancia"]
    .sum()
    .reset_index()
    .sort_values(by="Importancia", ascending=False)
)

# Visualización top 20
plt.figure(figsize=(8,10))
sns.barplot(
    x="Importancia", 
    y="Variable_grouped", 
    data=grouped.head(20),
    hue="Importancia", 
    dodge=False, 
    legend=False, 
    palette="Blues_r"
)
plt.title("Top 20 variables más importantes (XGBoost agrupado)")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.show()

# Mostrar tabla
display(grouped.head(20))

# Convertir top 10 a lista
top_features = grouped.head(10)["Variable_grouped"].tolist()
print("Top 10 variables agrupadas:", top_features)


## **Paso 11. Comparación: baseline vs. modelo optimizado**

Ya tenemos dos versiones del modelo:

1. **XGBoost baseline** → Entrenado con hiperparámetros por defecto.  
2. **XGBoost optimizado** → Ajustado con Optuna para minimizar el RMSE.  

Aquí comparamos ambas versiones en las tres métricas (RMSE, MAE y R²) para ver si la optimización realmente mejoró el rendimiento.  


In [ ]:
# ===================================
# 11. Comparación baseline vs. optimizado
# ===================================

# --- Baseline ---
xgb_base = XGBRegressor(random_state=42)
xgb_base.fit(X_train, y_train)
preds_base = xgb_base.predict(X_valid)

baseline_results = {
    "RMSE": np.sqrt(mean_squared_error(y_valid, preds_base)),
    "MAE": mean_absolute_error(y_valid, preds_base),
    "R2": r2_score(y_valid, preds_base)
}

# --- Optimizado con Optuna ---
xgb_best = XGBRegressor(**study.best_params, random_state=42)
xgb_best.fit(X_train, y_train)
preds_best = xgb_best.predict(X_valid)

optimized_results = {
    "RMSE": np.sqrt(mean_squared_error(y_valid, preds_best)),
    "MAE": mean_absolute_error(y_valid, preds_best),
    "R2": r2_score(y_valid, preds_best)
}

# --- Comparación ---
comparison_df = pd.DataFrame([baseline_results, optimized_results], 
                             index=["Baseline", "Optimizado"])

print("📊 Comparación de XGBoost baseline vs. optimizado:\n")
display(comparison_df.style.format({"RMSE": "{:.3f}", "MAE": "{:.3f}", "R2": "{:.3f}"})
        .background_gradient(cmap="Blues"))


# ===================================
# Función para guardar métricas estandarizadas
# ===================================
def save_results(model_name, y_true, y_pred, filepath):
    """
    Calcula métricas (RMSE, MAE, R²) y guarda en CSV estandarizado.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    results = pd.DataFrame([{
        "Modelo": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }])

    results.to_csv(filepath, index=False)
    print(f"✅ Resultados de {model_name} guardados en {filepath}")
    return results


# ===================================
# Guardar resultados en CSV
# ===================================

# ===================================
# Función para guardar métricas estandarizadas
# ===================================
def save_results(model_name, y_true, y_pred, filepath):
    """
    Calcula métricas (RMSE, MAE, R²) y guarda en CSV estandarizado.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    results = pd.DataFrame([{
        "Modelo": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }])

    results.to_csv(filepath, index=False)
    print(f"✅ Resultados de {model_name} guardados en {filepath}")
    return results



# ===================================
# Guardar métricas XGBoost (baseline)
# ===================================
results_xgb_base = pd.DataFrame([{
    "Modelo": "XGBoost (baseline)",
    "RMSE": baseline_results["RMSE"],
    "MAE": baseline_results["MAE"],
    "R²": baseline_results["R2"]
}])

results_xgb_base.to_csv("../data/results_xgboost_baseline.csv", index=False)

print("📊 XGBoost (baseline)")
print(f"RMSE: {baseline_results['RMSE']:.3f}")
print(f"MAE: {baseline_results['MAE']:.3f}")
print(f"R²: {baseline_results['R2']:.3f}")
print("✅ Resultados guardados en data/results_xgboost_baseline.csv")


# ===================================
# Guardar métricas XGBoost (Optuna / optimizado)
# ===================================
results_xgb_optuna = pd.DataFrame([{
    "Modelo": "XGBoost (optuna)",
    "RMSE": optimized_results["RMSE"],
    "MAE": optimized_results["MAE"],
    "R²": optimized_results["R2"]
}])

results_xgb_optuna.to_csv("../data/results_xgboost_optuna.csv", index=False)

print("📊 XGBoost (optuna)")
print(f"RMSE: {optimized_results['RMSE']:.3f}")
print(f"MAE: {optimized_results['MAE']:.3f}")
print(f"R²: {optimized_results['R2']:.3f}")
print("✅ Resultados guardados en data/results_xgboost_optuna.csv")



## **Paso 12. Conclusiones finales sobre XGBoost**

Tras entrenar y evaluar XGBoost en este dataset de esperanza de vida, podemos resumir:

- **Baseline**: el modelo ya ofrece un rendimiento sólido gracias a su naturaleza de boosting, que combina múltiples árboles secuenciales.  
- **Optimización con Optuna**: los hiperparámetros ajustados permitieron reducir el error (RMSE y MAE) y mejorar el R².  
  - Esto significa que el modelo se adapta mejor a la complejidad de los datos sin sobreajustarse.  
- **Interpretación del gráfico de convergencia**: observamos que el RMSE fue descendiendo en las sucesivas pruebas de Optuna hasta estabilizarse en un valor cercano al mínimo. Esto nos confirma que la búsqueda fue efectiva.  
- **Comparación global**:  
  - XGBoost supera a un árbol de decisión simple, que tiende a sobreajustar.  
  - También mejora a Random Forest en este dataset, ya que el boosting maneja mejor las interacciones complejas y reduce el sesgo.  
  - Frente a modelos lineales (como Regresión Lineal o Ridge), XGBoost tiene ventaja porque los datos no siguen relaciones puramente lineales.

✅ **Conclusión principal**:  
XGBoost optimizado es el modelo más robusto y preciso en este proyecto, y se justifica usarlo como **modelo final para el PMV**.  


In [ ]:
print("📌 Conclusiones:")
print("- Baseline XGBoost ya era competitivo, pero tras optimización bajó el RMSE y subió el R².")
print("- Optuna encontró hiperparámetros que reducen sobreajuste y capturan mejor la relación entre variables.")
print("- Frente a Decision Tree y Random Forest, XGBoost se adapta mejor a la complejidad del dataset.")
print("- Frente a modelos lineales, maneja relaciones no lineales y múltiples interacciones de forma más eficaz.")
print("👉 Modelo final recomendado: XGBoost optimizado.")
